In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
import matplotlib.pyplot as plt
import cv2
img = cv2.imread('/kaggle/input/face-mask-lite-dataset/with_mask/with-mask-default-mask-seed9854.png')
plt.imshow(img)

In [5]:
without = '/kaggle/input/face-mask-lite-dataset/without_mask'
withmask = '/kaggle/input/face-mask-lite-dataset/with_mask'
train = '/kaggle/input/face-mask-lite-dataset'

In [6]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [7]:
classes = ['with_mask','without_mask']
data =[]
for i in enumerate(os.listdir(train)):
    path = os.path.join(train,i[1])
    for j in os.listdir(path):
        img = cv2.imread(os.path.join(path,j),0)
        img = cv2.resize(img,(100,100))
        data.append([img,i[0]])

In [8]:
img=[]
label=[]
for x,y in data:
    img.append(x)
    label.append(y)
img = np.array(img).reshape(-1,100,100,1)
label = np.array(label)

In [9]:
from sklearn.model_selection import train_test_split

trainF,testF,trainl,testl = train_test_split(img,label,random_state=42,test_size=.2)

# Model


In [13]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D,SeparableConv2D,Flatten,Dense,MaxPooling2D


In [15]:
model = Sequential()
model.add(SeparableConv2D(32,kernel_size=3,padding='same',kernel_initializer ='glorot_uniform',input_shape = (100,100,1),activation='LeakyReLU'))
model.add(MaxPooling2D(strides=2))
model.add(SeparableConv2D(64,kernel_size=3,padding='same',kernel_initializer='glorot_uniform',activation='LeakyReLU'))
model.add(MaxPooling2D(strides=2))
model.add(SeparableConv2D(128,kernel_size=3,padding='same',kernel_initializer='glorot_uniform',activation='LeakyReLU'))
model.add(MaxPooling2D(strides=2))
model.add(SeparableConv2D(256,kernel_size=3,padding='same',kernel_initializer='glorot_uniform',activation='LeakyReLU'))
model.add(MaxPooling2D(strides=2))

model.add(SeparableConv2D(512,kernel_size=3,padding='same',kernel_initializer='glorot_uniform',activation='LeakyReLU'))
model.add(MaxPooling2D(strides=2))

model.add(Flatten())
model.add(Dense(4098,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [17]:
from tensorflow.keras.optimizers import Adam

In [19]:
model.compile(loss = 'binary_crossentropy',optimizer = Adam(.001),metrics=['accuracy'])

In [22]:
model.fit(trainF,trainl,validation_split=.2,epochs=5,batch_size=20)

In [31]:
pred = model.predict(testF)

for i in range(len(pred)):
    pred[i] = np.argmax(pred[i])

In [33]:
val = model.evaluate(testF,testl,steps=200)
print(val[1])

In [38]:
pred = np.array(model.predict(testF[:50]),dtype='int')

In [40]:
for i in range(len(pred)):
    print(pred[i],testl[i])

In [42]:
len(testF)

In [45]:
plt.figure(figsize=(15,15))
for i in enumerate(testF[:50]):
    plt.subplot(5,10,i[0]+1)
    plt.xlabel(str(pred[i[0]]))
    plt.imshow(i[1])